In [1]:
import math
import numpy as np
import torch
from datasets import load_dataset
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaTokenizerFast, RobertaConfig, RobertaModelWithHeads
from transformers import Trainer, TrainingArguments, EvalPrediction
from transformers import DataCollatorForLanguageModeling
from transformers import RobertaForMaskedLM
from transformers import AdapterType
from sklearn.metrics import f1_score
import datetime

In [2]:
today = datetime.date.today().strftime('%Y%m%d')

In [3]:
def check_gpu():
    '''
    check gpu status
    '''
    try:
        print('GPU available:', torch.cuda.is_available())
        print(torch.cuda.device_count(), 'GPUs detected')
        print('Current GPU id:', torch.cuda.current_device())
        print('Current GPU Name:', torch.cuda.get_device_name(torch.cuda.current_device()))
    except:
        print('GPU not available')
        
def encode_batch(batch):
    '''
    Encodes a batch of input data using the model tokenizer
    using 512
    '''
    return tokenizer(batch["text"], max_length=120, truncation=True, padding="max_length")
#     return tokenizer(batch["text"], padding=True, truncation=True)

def compute_accuracy(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {'acc': (preds==p.label_ids).mean()}

def compute_f1(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {'f1': f1_score(p.label_ids, preds, average=f1_type)}

def compute_score(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {'f1-micro': f1_score(p.label_ids, preds, average='micro'),'f1-macro': f1_score(p.label_ids, preds, average='macro'), 'acc': (preds==p.label_ids).mean()}

In [4]:
check_gpu()

GPU available: True
1 GPUs detected
Current GPU id: 0
Current GPU Name: GeForce RTX 2070 with Max-Q Design


In [5]:
# dictionary for dataset, name: (classes, type of f1 score)
dataset_dict = {'chemprot': (13, 'micro'), 'rct': (5, 'micro'),
                'CI': (6, 'macro'), 'sciie': (7, 'm2cro'),
                'HN': (2, 'macro'), 'ag': (4, 'macro'),
                'amazon': (2, 'macro'), 'imdb': (2, 'macro')}

In [6]:
ds_name = 'amazon'
n_labels = dataset_dict[ds_name][0]
f1_type = dataset_dict[ds_name][1]

In [7]:
dataset = load_dataset(f'data_loaders/{ds_name}_data_loader.py')

Reusing dataset task_dataset (C:\Users\19197\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae)


In [8]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

In [9]:
#dataset_encoded = dataset.map(encode_batch, batched=True, batch_size=512, remove_columns=["text"])
dataset_encoded = dataset.map(encode_batch, batched=True)
# tokenized_datasets = dataset.map(tokenizer, batched=True, num_proc=4, remove_columns=["text"])

Loading cached processed dataset at C:\Users\19197\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae\cache-294b1a221a18a2c0.arrow
Loading cached processed dataset at C:\Users\19197\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae\cache-bea7b60a230a39b6.arrow
Loading cached processed dataset at C:\Users\19197\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae\cache-c40cb088f0753106.arrow


In [10]:
dataset_encoded.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 115251
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 25000
    })
})

## Case 4
adapter language model

In [14]:
model = RobertaForMaskedLM.from_pretrained('roberta-base')

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model.add_adapter('mlm', AdapterType.text_lang)

In [16]:
model.set_active_adapters(['mlm'])

In [17]:
model.train_adapter(["mlm"])

In [18]:
# get warm up steps for given warmup ratio

warmup_ratio = 0.006
max_train_batch_size_mlm = 32
WARMUP_STEP = max(1,int(dataset_encoded['train'].num_rows / max_train_batch_size_mlm * warmup_ratio))
print(WARMUP_STEP)
GRADIENT_ACC_STEP = 256 / max_train_batch_size_mlm
print(GRADIENT_ACC_STEP)

21
8.0


In [19]:
training_args_mlm = TrainingArguments(
    output_dir=f'model/{ds_name}/mlm-adapter/{today}/',
    evaluation_strategy = "epoch",
    learning_rate=0.00025,
    remove_unused_columns=False,
    num_train_epochs=100,
    fp16=True,
    per_device_train_batch_size=max_train_batch_size_mlm,
    per_device_eval_batch_size=32,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
#     warmup_ratio=0.006, not supported in adapter-transformers
    warmup_steps=WARMUP_STEP,
    weight_decay=0.01,
    gradient_accumulation_steps=GRADIENT_ACC_STEP
)

In [20]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [28]:
trainer_mlm = Trainer(
    model=model,
    args=training_args_mlm,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"],
    data_collator=data_collator,
    compute_metrics=compute_score
)

In [22]:
trainer_mlm.train()

Epoch,Training Loss,Validation Loss
0,No log,1.589875
1,1.790488,1.562097
2,1.672804,1.556694
3,1.635920,1.526788
4,1.615113,1.495778
5,1.601512,1.496184
6,1.591165,1.481291
7,1.577594,1.494827
8,1.569970,1.469940
9,1.563271,1.497905


TrainOutput(global_step=45000, training_loss=1.487103125)

In [24]:
# eval_results = trainer.evaluate()
# print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [25]:
model.save_pretrained(f'model/amazon/{today}/model')

In [26]:
model.save_adapter(f'model/amazon/{today}/adapter', 'mlm')

## Baseline Finetune

In [13]:
model = RobertaModelWithHeads.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

In [14]:
model.add_classification_head('Amazon_classifier', num_labels=n_labels)

In [15]:
# get warm up steps for given warmup ratio

warmup_ratio = 0.006
max_train_batch_size_mlm = 16
WARMUP_STEP = max(1, int(dataset_encoded['train'].num_rows / max_train_batch_size_mlm * warmup_ratio))
print(WARMUP_STEP)
GRADIENT_ACC_STEP = 256 / max_train_batch_size_mlm
print(GRADIENT_ACC_STEP)

43
16.0


In [17]:
trainer_ft = Trainer(
    model=model,
    args=training_args_ft,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"],
    compute_metrics= compute_score
#     data_collator=data_collator,
)

In [16]:
training_args_ft = TrainingArguments(
    output_dir=f'model/{ds_name}/ft/{today}/',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    remove_unused_columns=False,
    num_train_epochs=3,
    fp16=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
#     warmup_ratio=0.006, not supported in adapter-transformers
    warmup_steps=WARMUP_STEP,
    weight_decay=0.01,
    gradient_accumulation_steps=GRADIENT_ACC_STEP
)

In [18]:
trainer_ft.train()

Epoch,Training Loss,Validation Loss,F1-micro,F1-macro,Acc
0,No log,0.312540,0.868800,0.663893,0.868800
1,0.339165,0.305352,0.871200,0.643489,0.871200
2,0.294193,0.310035,0.874600,0.665557,0.874600


TrainOutput(global_step=1350, training_loss=0.30662575050636576)

In [19]:
trainer_ft.evaluate()

{'eval_loss': 0.3100349009037018,
 'eval_f1-micro': 0.8746,
 'eval_f1-macro': 0.6655568702139828,
 'eval_acc': 0.8746,
 'epoch': 2.999444752915047}

In [20]:
trainer_ft.evaluate(dataset_encoded['test'])

{'eval_loss': 0.3098597824573517,
 'eval_f1-micro': 0.87452,
 'eval_f1-macro': 0.6588806506844302,
 'eval_acc': 0.87452,
 'epoch': 2.999444752915047}

## Pretrained Baseline TAPT

In [21]:
model = RobertaModelWithHeads.from_pretrained('allenai/dsp_roberta_base_tapt_amazon_helpfulness_115K')

Some weights of the model checkpoint at allenai/dsp_roberta_base_tapt_amazon_helpfulness_115K were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at allenai/dsp_roberta_base_tapt_amazon_helpfulness_115K and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAI

In [22]:
model.add_classification_head('Amazon_classifier', num_labels=n_labels)

In [23]:
# get warm up steps for given warmup ratio

warmup_ratio = 0.006
max_train_batch_size_mlm = 16
WARMUP_STEP = max(1, int(dataset_encoded['train'].num_rows / max_train_batch_size_mlm * warmup_ratio))
print(WARMUP_STEP)
GRADIENT_ACC_STEP = 256 / max_train_batch_size_mlm
print(GRADIENT_ACC_STEP)

43
16.0


In [24]:
trainer_ft = Trainer(
    model=model,
    args=training_args_ft,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"],
    compute_metrics= compute_score
#     data_collator=data_collator,
)

In [25]:
training_args_ft = TrainingArguments(
    output_dir=f'model/{ds_name}/pt-ft/{today}/',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    remove_unused_columns=False,
    num_train_epochs=3,
    fp16=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
#     warmup_ratio=0.006, not supported in adapter-transformers
    warmup_steps=WARMUP_STEP,
    weight_decay=0.01,
    gradient_accumulation_steps=GRADIENT_ACC_STEP
)

In [26]:
trainer_ft.train()

Epoch,Training Loss,Validation Loss,F1-micro,F1-macro,Acc
0,No log,0.297437,0.879200,0.683938,0.879200
1,0.321902,0.296483,0.881800,0.695489,0.881800
2,0.272898,0.298727,0.879800,0.697147,0.879800


TrainOutput(global_step=1350, training_loss=0.28658112702546296)

In [27]:
trainer_ft.evaluate()

{'eval_loss': 0.29872748255729675,
 'eval_f1-micro': 0.8798,
 'eval_f1-macro': 0.6971471666227758,
 'eval_acc': 0.8798,
 'epoch': 2.999444752915047}

In [28]:
trainer_ft.evaluate(dataset_encoded['test'])

{'eval_loss': 0.29575395584106445,
 'eval_f1-micro': 0.8817200000000001,
 'eval_f1-macro': 0.693303043249042,
 'eval_acc': 0.88172,
 'epoch': 2.999444752915047}

## Adapter Pretrain Finetune

In [12]:
model = RobertaModelWithHeads.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

In [13]:
model.load_adapter('model/amazon/20210429/adapter')

'mlm'

In [14]:
model.set_active_adapters(['mlm'])

In [15]:
model.add_classification_head('Amazon_classifier', num_labels=n_labels)

In [16]:
# get warm up steps for given warmup ratio

warmup_ratio = 0.006
max_train_batch_size_mlm = 16
WARMUP_STEP = max(1, int(dataset_encoded['train'].num_rows / max_train_batch_size_mlm * warmup_ratio))
print(WARMUP_STEP)
# GRADIENT_ACC_STEP = 256 / max_train_batch_size_mlm
# print(GRADIENT_ACC_STEP)

43


In [17]:
training_args_ft = TrainingArguments(
    output_dir=f'model/{ds_name}/adapter-pt-ft/{today}/',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    remove_unused_columns=False,
    num_train_epochs=3,
    fp16=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
#     warmup_ratio=0.006, not supported in adapter-transformers
    warmup_steps=WARMUP_STEP,
    weight_decay=0.01,
#     gradient_accumulation_steps=GRADIENT_ACC_STEP
)

In [18]:
trainer_ft = Trainer(
    model=model,
    args=training_args_ft,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"],
    compute_metrics=compute_score
#     data_collator=data_collator,
)

In [ ]:
trainer_ft.train()

Epoch,Training Loss,Validation Loss


In [ ]:
trainer_ft.evaluate()

In [ ]:
trainer_ft.evaluate(dataset_encoded['test'])

## Adapter Pretrain Adapter Finetune

In [12]:
model = RobertaModelWithHeads.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

In [13]:
model.load_adapter('model/amazon/20210429/adapter')

'mlm'

In [14]:
model.add_classification_head('Amazon_classifier', num_labels=n_labels)

In [15]:
model.add_adapter('Amazon_classifier', adapter_type=AdapterType.text_task)

In [17]:
model.set_active_adapters([['mlm', 'Amazon_classifier']])

In [16]:
model.train_adapter(['Amazon_classifier'])

In [18]:
# get warm up steps for given warmup ratio

warmup_ratio = 0.006
max_train_batch_size_mlm = 16
WARMUP_STEP = max(1, int(dataset_encoded['train'].num_rows / max_train_batch_size_mlm * warmup_ratio))
print(WARMUP_STEP)
# GRADIENT_ACC_STEP = 256 / max_train_batch_size_mlm
# print(GRADIENT_ACC_STEP)

1
16.0


In [19]:
training_args_ft = TrainingArguments(
    output_dir=f'model/{ds_name}/adapter-pt-adapter-ft/{today}/',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    remove_unused_columns=False,
    num_train_epochs=3,
    fp16=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
#     warmup_ratio=0.006, not supported in adapter-transformers
    warmup_steps=WARMUP_STEP,
    weight_decay=0.01,
#     gradient_accumulation_steps=GRADIENT_ACC_STEP
)

In [20]:
trainer_ft = Trainer(
    model=model,
    args=training_args_ft,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"],
    compute_metrics= compute_score
#     data_collator=data_collator,
)

In [21]:
trainer_ft.train()

Epoch,Training Loss,Validation Loss,F1-micro,F1-macro,Acc
0,No log,1.805332,0.043860,0.014006,0.043860
1,No log,1.782240,0.149123,0.064966,0.149123
2,No log,1.773764,0.517544,0.114341,0.517544


TrainOutput(global_step=18, training_loss=1.9204243554009333)

In [22]:
trainer_ft.evaluate()

{'eval_loss': 1.7737641334533691,
 'eval_f1-micro': 0.5175438596491229,
 'eval_f1-macro': 0.11434108527131782,
 'eval_acc': 0.5175438596491229,
 'epoch': 2.9056603773584904}

In [23]:
trainer_ft.evaluate(dataset_encoded['test'])

{'eval_loss': 1.7735462188720703,
 'eval_f1-micro': 0.5107913669064749,
 'eval_f1-macro': 0.1126984126984127,
 'eval_acc': 0.5107913669064749,
 'epoch': 2.9056603773584904}